In [6]:
import requests
import os
import json
from dotenv import load_dotenv
import gradio as gr 

In [7]:
load_dotenv()

api_keys={ "OPEN_AI_API_KEY":"sk-proj-",
    "ANTHROPIC_API_KEY": "sk-ant-",
    "GOOGLE_GEMINI_API_KEY": "gems-"}

def validate_key(name,prefix):
    key = os.getenv(name)
    if key and key.startswith(prefix):
        print(f"✅ {name} is valid.")
    elif key:
        print(f"⚠️ {name} is present but may be malformed.")
    else:
        print(f"❌ {name} is missing from environment.")
    return key


openai_key = validate_key("OPEN_AI_API_KEY", api_keys["OPEN_AI_API_KEY"])
anthropic_key = validate_key("ANTHROPIC_API_KEY", api_keys["ANTHROPIC_API_KEY"])
gemini_key = validate_key("GOOGLE_GEMINI_API_KEY", api_keys["GOOGLE_GEMINI_API_KEY"])

if openai_key:
    import openai
    openai.api_key = openai_key

if anthropic_key:
    from anthropic import Anthropic
    client = Anthropic(api_key=anthropic_key)


if gemini_key:
    import google.generativeai as genai
    genai.configure(api_key=gemini_key)

    os.environ["GOOGLE_GEMINI_API_KEY"] = gemini_key
    

✅ OPEN_AI_API_KEY is valid.
✅ ANTHROPIC_API_KEY is valid.
⚠️ GOOGLE_GEMINI_API_KEY is present but may be malformed.


In [34]:
MODEL = "gpt-4o-mini"

In [35]:
system_message = "You are a helpful assistant for an Airline called FlightAI."
system_message += "Give short, courteous answer, no more that 1 sentence"
system_message += "Always be accurate. If you don't know the answer, say so."

In [36]:
def chat(message, history):
    messages = [{"role":"user", "content":system_message}]
    for human, assistant in history:
        messages.append({"role":"user","content":human})
        messages.append({"role":"assistant","content":assistant})
    messages.append({"role":"user","content":message})

    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content
# When you want to Stream your response 
    # stream = openai.chat.completions.create(model="chatgpt-4o-latest", messages=messages, stream=True)
    # response = ""
    # for chunk in stream:
    #     response += chunk.choices[0].delta.content or ""
    #     yield response

In [37]:
gr.ChatInterface(fn=chat).launch()

c:\Users\mathe\OneDrive\Desktop\LLM_Course\Large_Language_Model_course_Material\Week2\.venv\lib\site-packages\gradio\chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [27]:
ticket_prices = {"london":"$799", "paris":"$899", "tokyo":"$1400","berlin":"$499"}

def get_ticket_prices(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")
    

In [28]:
get_ticket_prices("paris")

Tool get_ticket_price called for paris


'$899'

In [29]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [30]:
tools = [{"type":"function", "function":price_function}]

In [31]:
def chat(message, history):
    messages = [{"role":"user", "content":system_message}]
    for human, assistant in history:
        messages.append({"role":"user","content":human})
        messages.append({"role":"assistant","content":assistant})
    messages.append({"role":"user","content":message})

    response = openai.chat.completions.create(model="chatgpt-4o-latest", messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        message.append(message)
        message.append(response)
        response = openai.chat.completions.create(model="chatgpt-4o-latest",messages=messages, tools=tools)
    return response.choices[0].message.content

In [32]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.load(tool_call.function.arguments)
    city = arguments.get("destination_city")
    price = get_ticket_prices(city)
    response = {
        "role":"tool",
        "content": json.dumps({"destination_city":city,"price":price}),"tool_call_id":message.tool_calls[0].id}
    return response, city

In [33]:
gr.ChatInterface(fn=chat).launch()

c:\Users\mathe\OneDrive\Desktop\LLM_Course\Large_Language_Model_course_Material\Week2\.venv\lib\site-packages\gradio\chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\mathe\OneDrive\Desktop\LLM_Course\Large_Language_Model_course_Material\Week2\.venv\lib\site-packages\gradio\queueing.py", line 626, in process_events
    response = await route_utils.call_process_api(
  File "c:\Users\mathe\OneDrive\Desktop\LLM_Course\Large_Language_Model_course_Material\Week2\.venv\lib\site-packages\gradio\route_utils.py", line 349, in call_process_api
    output = await app.get_blocks().process_api(
  File "c:\Users\mathe\OneDrive\Desktop\LLM_Course\Large_Language_Model_course_Material\Week2\.venv\lib\site-packages\gradio\blocks.py", line 2274, in process_api
    result = await self.call_function(
  File "c:\Users\mathe\OneDrive\Desktop\LLM_Course\Large_Language_Model_course_Material\Week2\.venv\lib\site-packages\gradio\blocks.py", line 1779, in call_function
    prediction = await fn(*processed_input)
  File "c:\Users\mathe\OneDrive\Desktop\LLM_Course\Large_Language_Model_course_Material\Week2\.venv\lib\site-packag